In [104]:
import os
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import tkinter as tk

In [105]:
load_dotenv()

True

In [106]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = os.getenv("BASE_URL")

In [107]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2"
)

In [ ]:
# Set the stock tickers
symbols = ['PANW', 'LRCX', 'AAPL', 'AMZN', 'MSFT', 'TSLA', 'CAT', 'DE']

In [108]:
# Define trading strategy
def trading_strategy(symbol, quantity):
    # Set timeframe to "1Day"
    timeframe = "1D"

In [110]:
# Set start and end datetimes of 1 month, between now and 1 month ago.
start_date = pd.Timestamp("2023-06-10", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-07-10", tz="America/New_York").isoformat()

In [ ]:
# Get historical data for the specified symbol
historical_data = api.get_bars(
    symbol,
    timeframe,
    start=start_date,
    end=end_date
    ).df[symbol]

if not historical_data.empty:
    # Calculate SMA indicators
    historical_data['SMA30'] = historical_data['close'].rolling(window=30).mean()
    historical_data['SMA50'] = historical_data['close'].rolling(window=50).mean()

In [ ]:
# Determine the latest SMA values
latest_sma30 = historical_data['SMA30'].iloc[-1]
latest_sma50 = historical_data['SMA50'].iloc[-1]

In [ ]:
# Check for trading signals
if latest_sma30 > latest_sma50:
    # Example: Buy shares of the specified stock if SMA30 crosses above SMA50
    qty = int(quantity)
    side = 'buy'
    order_type = 'market'
    
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force='gtc'
    )
    result_label.config(text='Buy order submitted.')
           
elif latest_sma30 < latest_sma50:
    # Example: Sell all shares of the specified stock if SMA30 crosses below SMA50
    qty = 'all'
    side = 'sell'
    order_type = 'market'
    
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force='gtc'
    )
    result_label.config(text='Sell order submitted.')
           

            api.submit_order(
                symbol=symbol,
                qty=qty,
                side=side,
                type=order_type,
                time_in_force='gtc'
            )
            result_label.config(text='Sell order submitted.')
            

else:
    result_label.config(text='No trading signals.')

    
else:
    result_label.config(text='Symbol not found or no historical data available.')       

In [ ]:
# Function to handle button click
def submit_button_click():
    symbol = symbol_entry.get().upper()
    quantity = quantity_entry.get()
    trading_strategy(symbol, quantity)

In [ ]:
# Create GUI window
window = tk.Tk()
window.title('Alpaca Trading Bot')
window.geometry('400x250')

In [ ]:
# Create input fields and labels
symbol_label = tk.Label(window, text='Symbol:')
symbol_label.pack()
symbol_entry = tk.Entry(window)
symbol_entry.pack()

quantity_label = tk.Label(window, text='Quantity:')
quantity_label.pack()
quantity_entry = tk.Entry(window)
quantity_entry.pack()

In [ ]:
# Create result label
result_label = tk.Label(window, text='')
result_label.pack()

In [ ]:
# Create submit button
submit_button = tk.Button(window, text='Submit', command=submit_button_click)
submit_button.pack()

In [ ]:
window.mainloop()